## ML-uppgift

### Vi börjar med att importera bibliotek och läsa in filen. 

In [104]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from sklearn.model_selection import KFold


In [105]:

df = pd.read_csv('car_price_dataset.csv', sep=";")  


I mitt experimenterande visade det sig att separatorn i CSV-filen är ; och inte , - fixade det ovan.

Hur ser dataunderlaget ut?

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Brand         10000 non-null  object 
 1   Model         10000 non-null  object 
 2   Year          10000 non-null  int64  
 3   Engine_Size   10000 non-null  float64
 4   Fuel_Type     10000 non-null  object 
 5   Transmission  10000 non-null  object 
 6   Mileage       10000 non-null  int64  
 7   Doors         10000 non-null  int64  
 8   Owner_Count   10000 non-null  int64  
 9   Price         10000 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 781.4+ KB


In [107]:
print(df.isnull().sum())

Brand           0
Model           0
Year            0
Engine_Size     0
Fuel_Type       0
Transmission    0
Mileage         0
Doors           0
Owner_Count     0
Price           0
dtype: int64


Inga null-värden

In [108]:
print(df.describe())
print(df.select_dtypes(include='object').nunique())


               Year   Engine_Size        Mileage         Doors   Owner_Count  \
count  10000.000000  10000.000000   10000.000000  10000.000000  10000.000000   
mean    2011.543700      3.000560  149239.111800      3.497100      2.991100   
std        6.897699      1.149324   86322.348957      1.110097      1.422682   
min     2000.000000      1.000000      25.000000      2.000000      1.000000   
25%     2006.000000      2.000000   74649.250000      3.000000      2.000000   
50%     2012.000000      3.000000  149587.000000      3.000000      3.000000   
75%     2017.000000      4.000000  223577.500000      4.000000      4.000000   
max     2023.000000      5.000000  299947.000000      5.000000      5.000000   

             Price  
count  10000.00000  
mean    8852.96440  
std     3112.59681  
min     2000.00000  
25%     6646.00000  
50%     8858.50000  
75%    11086.50000  
max    18301.00000  
Brand           10
Model           30
Fuel_Type        4
Transmission     3
dtype: int64


Alla rader är unika, inga dubletter och inga värden saknas. Det finns tio bilmärken, 30 olika modeller, fyra bränsletyper och tre växellådor. Hur kan det finnas tre olika växellådor frågar sig vän av ordning? Vilka är det? Vi kollar.

In [109]:
print(df.Transmission.unique())

['Manual' 'Automatic' 'Semi-Automatic']


Frågan är hur semi-automatic och automatic skiljer sig? Kanske ska man slå ihop dessa till en så man får manuell eller automatisk växellåda? Vi kollar om det där semi-automatic kanske bara är brus som förvirrar modellen?


In [110]:
trans_pct = df["Transmission"].value_counts(normalize=True) * 100
print(trans_pct.round(1))


Transmission
Manual            33.7
Automatic         33.2
Semi-Automatic    33.1
Name: proportion, dtype: float64


Nej, det är en tredjedel av varje. Märkligt, men jag lämnar detta beslut tills jag ser hur priset korrelerar med de olika alternativen.

Först gör vi dock en enkel koll på de numeriska kolumnerna och hur de korrelererar med priset. 

In [111]:
numeric_cols = ["Year", "Engine_Size", "Mileage", "Doors", "Owner_Count", "Price"]
print(df[numeric_cols].corr()["Price"].sort_values(ascending=False))


Price          1.000000
Year           0.663036
Engine_Size    0.357403
Owner_Count    0.002656
Doors          0.000511
Mileage       -0.551227
Name: Price, dtype: float64


Och så de textbaserade kolumnerna. Vi stryker Brand ur hanteringen eftersom det ju implicit bestäms av Model

In [112]:
cats = ["Model", "Fuel_Type", "Transmission"]

# 1. Dummies
X_cat = pd.get_dummies(df[cats], drop_first=True)

# 2. Korrelationer för alla dummies mot Price
corrs = X_cat.join(df["Price"]).corr()["Price"]

# 3. Summera absoluta korrelationer per variabel
model_corr        = corrs[corrs.index.str.startswith("Model_")].abs().sum()
fuel_type_corr    = corrs[corrs.index.str.startswith("Fuel_Type_")].abs().sum()
transmission_corr = corrs[corrs.index.str.startswith("Transmission_")].abs().sum()

summary = pd.Series(
    {
        "Model": model_corr,
        "Fuel_Type": fuel_type_corr,
        "Transmission": transmission_corr,
    },
    name="Total_abs_corr_with_Price"
)

print(summary)


Model           0.219170
Fuel_Type       0.415330
Transmission    0.245259
Name: Total_abs_corr_with_Price, dtype: float64


Nu blir jag ändå lite misstänksam mot växellådorna där. Jag får kolla lite mer noga. 

In [113]:
print(df.groupby("Transmission")["Price"].describe())

med_iqr = df.groupby("Transmission")["Price"].agg(
    median="median",
    q1=lambda s: s.quantile(0.25),
    q3=lambda s: s.quantile(0.75)
)
print(med_iqr)


                 count         mean          std     min     25%      50%  \
Transmission                                                                
Automatic       3317.0  9938.252939  3025.692193  2000.0  7784.0  10034.0   
Manual          3372.0  8363.426157  3019.758983  2000.0  6159.5   8341.5   
Semi-Automatic  3311.0  8264.266385  3006.190812  2000.0  6069.0   8263.0   

                    75%      max  
Transmission                      
Automatic       12070.0  18301.0  
Manual          10514.5  16844.0  
Semi-Automatic  10364.5  17078.0  
                 median      q1       q3
Transmission                            
Automatic       10034.0  7784.0  12070.0
Manual           8341.5  6159.5  10514.5
Semi-Automatic   8263.0  6069.0  10364.5


Ajdå - semi-automatic är alltså en informativ feature ändå. Ja, ja - då får den vara kvar. 

## Dags att förbereda för träning


Det första vi ska göra är att ta itu med Doors och Owner_Count. Tillför dessa något eller är de mest brus i vårt dataset? 

Vi kör ett feature-set med alla variabler (fortfarande utan brand som vi ju valt bort som implicit bestämt av Model), ett utan doors och owner count, samt ett utan doors respektive owner count. 

In [114]:
base_cols = ["Model", "Year", "Engine_Size", "Fuel_Type",
             "Transmission", "Mileage"]

# Alla variabler
cols_all = base_cols + ["Doors", "Owner_Count"]

# Utan båda
cols_none = base_cols

# Utan Doors (men med Owner_Count)
cols_no_doors = base_cols + ["Owner_Count"]

# Utan Owner_Count (men med Doors)
cols_no_owner = base_cols + ["Doors"]


Vi gör en test med Lasso (endast i syfte att granska relevansen för de två kolumnerna).


In [115]:
def make_X(df, cols):
    X = df[cols]
    return pd.get_dummies(
        X,
        columns=["Model", "Fuel_Type", "Transmission"],
        drop_first=True
    )

def rmse_lasso(X_enc, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X_enc, y, test_size=0.2, random_state=42
    )
    model = Lasso(alpha=0.1, max_iter=10000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))


Och så ska vi se hur RMSE påverkas.

In [116]:
y = df["Price"]

rmse_all      = rmse_lasso(make_X(df, cols_all), y)
rmse_none     = rmse_lasso(make_X(df, cols_none), y)
rmse_no_doors = rmse_lasso(make_X(df, cols_no_doors), y)
rmse_no_owner = rmse_lasso(make_X(df, cols_no_owner), y)

print("RMSE alla features:        ", rmse_all)
print("RMSE utan Doors/Owner:     ", rmse_none)
print("RMSE utan Doors (med Owner)", rmse_no_doors)
print("RMSE utan Owner (med Doors)", rmse_no_owner)


RMSE alla features:         64.73418582536543
RMSE utan Doors/Owner:      64.7130558287325
RMSE utan Doors (med Owner) 64.71547309807511
RMSE utan Owner (med Doors) 64.73198122735877


Det visar alltså att RMSE inte påverkas mer än i liten utsträckning av Doors/Owner, varför dessa slopas i genomgången av modeller.

## Dags att testa modeller

Vi har valt att dela upp modellerna så att alla får arbeta med både linjär och trädbaserad modell. Här följer genomgång av Lasso, ElasticNet och HistGradientBoostingRegressor. Om tid finns följer även KNeighborsRegressor. 

En gemensam feature-set för alla modeller.

In [117]:
features = ["Model", "Year", "Engine_Size", "Fuel_Type",
            "Transmission", "Mileage"]
X = df[features]
y = df["Price"]

X_enc = pd.get_dummies(
    X,
    columns=["Model", "Fuel_Type", "Transmission"],
    drop_first=True
)


Så splittar vi upp data för träning, validering och test. Jag sparar 20 % för testningen för att få rimlig utvärdering av modellerna.

In [118]:
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X_enc, y, test_size=0.2, random_state=42
)


# Lasso



In [119]:
lasso_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),  
    ("model", Lasso(max_iter=10000))
])

# Hyperparametrar 
lasso_param_grid = {
    "model__alpha": [0.01, 0.1, 1.0]
}

# GridSearchCV 
lasso_grid = GridSearchCV(
    lasso_pipe,
    param_grid=lasso_param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

lasso_grid.fit(X_trainval, y_trainval)

print("Bästa alpha:", lasso_grid.best_params_)
print("CV-RMSE:", -lasso_grid.best_score_)


Bästa alpha: {'model__alpha': 1.0}
CV-RMSE: 89.00034192551473


Alpha ska alltså vara 1.0 och RMSE för min korsvalidering är 89. 

In [120]:
best_lasso = lasso_grid.best_estimator_

y_pred_test = best_lasso.predict(X_test)
lasso_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print("Test-RMSE Lasso:", lasso_rmse_test)


Test-RMSE Lasso: 64.89775587865122


Märkligt nog är RMSE för mina 20% testdata betydligt lägre - jag får dubbelkolla detta.

In [121]:
y_pred_trainval = best_lasso.predict(X_trainval)
rmse_trainval = np.sqrt(mean_squared_error(y_trainval, y_pred_trainval))
print("Train/val-RMSE (med best_lasso):", rmse_trainval)

Train/val-RMSE (med best_lasso): 90.25184605693242


Hmm. Skumt. Kan randomiseringen ha blivit så skum? Jag får köra en extra omgång med annat random-värde. 

För att inte störa hanteringen kommenterar jag bort detta nu efter testen. Resultaten är inlagda i kodrutan.

In [122]:
""" X_trainval, X_test, y_trainval, y_test = train_test_split(
    X_enc, y, test_size=0.2, random_state=123  
)

lasso_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),  
    ("model", Lasso(max_iter=10000))
])

# Hyperparametrar 
lasso_param_grid = {
    "model__alpha": [0.01, 0.1, 1.0, 10]
}

# GridSearchCV 
lasso_grid = GridSearchCV(
    lasso_pipe,
    param_grid=lasso_param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

lasso_grid.fit(X_trainval, y_trainval)

print("Bästa alpha:", lasso_grid.best_params_)
print("CV-RMSE:", -lasso_grid.best_score_)

best_lasso = lasso_grid.best_estimator_

y_pred_test = best_lasso.predict(X_test)
lasso_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print("Test-RMSE Lasso:", lasso_rmse_test)

"""

# Resultat
# Bästa alpha: {'model__alpha': 1.0}
# CV-RMSE: 85.59669801400972
# Test-RMSE Lasso: 70.96541649575882

' X_trainval, X_test, y_trainval, y_test = train_test_split(\n    X_enc, y, test_size=0.2, random_state=123  \n)\n\nlasso_pipe = Pipeline([\n    ("scaler", StandardScaler(with_mean=False)),  \n    ("model", Lasso(max_iter=10000))\n])\n\n# Hyperparametrar \nlasso_param_grid = {\n    "model__alpha": [0.01, 0.1, 1.0, 10]\n}\n\n# GridSearchCV \nlasso_grid = GridSearchCV(\n    lasso_pipe,\n    param_grid=lasso_param_grid,\n    scoring="neg_root_mean_squared_error",\n    cv=5,\n    n_jobs=-1\n)\n\nlasso_grid.fit(X_trainval, y_trainval)\n\nprint("Bästa alpha:", lasso_grid.best_params_)\nprint("CV-RMSE:", -lasso_grid.best_score_)\n\nbest_lasso = lasso_grid.best_estimator_\n\ny_pred_test = best_lasso.predict(X_test)\nlasso_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))\n\nprint("Test-RMSE Lasso:", lasso_rmse_test)\n\n'

Fortfarande en skum skillnad. Har mina data extremvärden som stör? 

df[["Price", "Year", "Engine_Size", "Mileage"]].describe()


|        |   Price     |    Year      | Engine_Size  |   Mileage     |
|--------|-------------|--------------|--------------|---------------|
| count  | 10000.00000 | 10000.000000 | 10000.000000 | 10000.000000  |
| mean   | 8852.96440  | 2011.543700  | 3.000560     | 149239.111800 |
| std    | 3112.59681  | 6.897699     | 1.149324     | 86322.348957  |
| min    | 2000.00000  | 2000.000000  | 1.000000     | 25.000000     |
| 25%    | 6646.00000  | 2006.000000  | 2.000000     | 74649.250000  |
| 50%    | 8858.50000  | 2012.000000  | 3.000000     | 149587.000000 |
| 75%    | 11086.50000 | 2017.000000  | 4.000000     | 223577.500000 |
| max    | 18301.00000 | 2023.000000  | 5.000000     | 299947.000000 |



Nej, det ser inte ut att finnas några extremt avvikande värden. 1,5×IQR‑regeln antyder att data är relativt jämnt fördelade i rimliga intervall. Skillnaden mellan CV‑RMSE och test‑RMSE beror därför troligen på en naturlig variation mellan i det här datasetet

# Elasticnet


In [123]:
from sklearn.linear_model import ElasticNet

elastic_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("model", ElasticNet(max_iter=10000))
])

elastic_param_grid = {
    "model__alpha": [0.01, 0.1, 1.0, 10],
    "model__l1_ratio": [0.2, 0.5, 0.8, 1.0]  
}

elastic_grid = GridSearchCV(
    elastic_pipe,
    param_grid=elastic_param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

elastic_grid.fit(X_trainval, y_trainval)

print("Bästa hyperparametrar:", elastic_grid.best_params_)
print("CV-RMSE ElasticNet:", -elastic_grid.best_score_)



Bästa hyperparametrar: {'model__alpha': 1.0, 'model__l1_ratio': 1.0}
CV-RMSE ElasticNet: 89.00034192551473


In [124]:
best_elastic = elastic_grid.best_estimator_

y_pred_test_elastic = best_elastic.predict(X_test)
elastic_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test_elastic))
print("Test-RMSE ElasticNet:", elastic_rmse_test)


Test-RMSE ElasticNet: 64.89775587865122


Även ElasticNet har ett tydligt högre CV‑RMSE än test‑RMSE. Gissningvis är det korsvalideringen som ger en mer försiktig uppskattning av modellens fel medan de 20% testdata är lättare att förutsäga. Modellen är inte överanpassad till testdatan, snarare är CV‑värdet konservativt.

# HistGradientBoostingRegressor


In [125]:
hgb_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),  
    ("model", HistGradientBoostingRegressor(random_state=123))
])

hgb_param_grid = {
    "model__learning_rate": [0.05, 0.1, 0.2], 
    "model__max_depth": [None, 3, 5], 
    "model__max_iter": [200, 300, 500],
    "model__min_samples_leaf": [20, 50, 100]
}

hgb_grid = GridSearchCV(
    hgb_pipe,
    param_grid=hgb_param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

hgb_grid.fit(X_trainval, y_trainval)

print("Bästa hyperparametrar:", hgb_grid.best_params_)
print("CV-RMSE HGB:", -hgb_grid.best_score_)





Bästa hyperparametrar: {'model__learning_rate': 0.1, 'model__max_depth': None, 'model__max_iter': 500, 'model__min_samples_leaf': 20}
CV-RMSE HGB: 135.37410096687785


Och så testar vi. 

In [126]:
best_hgb = hgb_grid.best_estimator_

y_pred_test_hgb = best_hgb.predict(X_test)
hgb_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test_hgb))
print("Test-RMSE HGB:", hgb_rmse_test)


Test-RMSE HGB: 120.6882800529531


Återigen är modellen bättre på testdata än tränings/valideringsdata. Ett RMSE på 135 visar ändå att det här *inte* är rätt modell. 

# KNeighborsRegressor

In [127]:

knn_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),  
    ("model", KNeighborsRegressor())
])

knn_param_grid = {
    "model__n_neighbors": [5, 10, 20, 40, 80],
    "model__weights": ["uniform", "distance"],
    "model__p": [1, 2]  
}

knn_grid = GridSearchCV(
    knn_pipe,
    param_grid=knn_param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

knn_grid.fit(X_trainval, y_trainval)

print("Bästa hyperparametrar:", knn_grid.best_params_)
print("CV-RMSE KNN:", -knn_grid.best_score_)



Bästa hyperparametrar: {'model__n_neighbors': 5, 'model__p': 1, 'model__weights': 'distance'}
CV-RMSE KNN: 1326.7581769989629


Sjukt dålig modell för de här data, men vi verifierar ändå mot testdata.

In [128]:
best_knn = knn_grid.best_estimator_

y_pred_test_knn = best_knn.predict(X_test)
knn_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test_knn))
print("Test-RMSE KNN:", knn_rmse_test)


Test-RMSE KNN: 1228.0529886564482


# Resultat
| Modell                          | CV‑RMSE | Test‑RMSE |
|---------------------------------|--------:|----------:|
| Lasso                           | 85      | 65        |
| ElasticNet                      | 86      | 72        |
| HistGradientBoostingRegressor   | 137     | 135       |
| KNeighborsRegressor             | 1339    | 1228      |

Utifrån RMSE‑resultaten verkar de linjära modellerna (Lasso och ElasticNet) vara klart mest lämpliga för det här datasetet, medan trädmodellerna HistGradientBoostingRegressor och KNeighborsRegressor ger betydligt större fel trots att jag suttit och försökt finslipa hyperparametrarna.

Man blir nästan lite misstänksam när den enklaste(?) modellen ger ett så litet fel som 65. Men det är väl bara att tacka och ta emot ... :D§

# Bonus
Jag provar ett par modeller till innan jag lägger av. 


# Ridge


In [129]:
from sklearn.linear_model import Ridge

ridge_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("model", Ridge(max_iter=10000))
])

ridge_param_grid = {
    "model__alpha": [0.01, 0.1, 1.0, 10, 100]
}

ridge_grid = GridSearchCV(
    ridge_pipe,
    param_grid=ridge_param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

ridge_grid.fit(X_trainval, y_trainval)

print("Bästa hyperparametrar Ridge:", ridge_grid.best_params_)
print("CV-RMSE Ridge:", -ridge_grid.best_score_)


Bästa hyperparametrar Ridge: {'model__alpha': 0.01}
CV-RMSE Ridge: 89.04761952060953


In [130]:

best_ridge = ridge_grid.best_estimator_

y_pred_test_ridge = best_ridge.predict(X_test)
ridge_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test_ridge))
print("Test-RMSE Ridge:", ridge_rmse_test)


Test-RMSE Ridge: 64.8909381793755


# RandomForestRegressor

In [131]:
from sklearn.ensemble import RandomForestRegressor

rf_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),  
    ("model", RandomForestRegressor(random_state=123))
])

rf_param_grid = {
    "model__n_estimators": [100, 300],
    "model__max_depth": [None, 10, 20],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 4]
}

rf_grid = GridSearchCV(
    rf_pipe,
    param_grid=rf_param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

rf_grid.fit(X_trainval, y_trainval)

print("Bästa hyperparametrar RF:", rf_grid.best_params_)
print("CV-RMSE RF:", -rf_grid.best_score_)




Bästa hyperparametrar RF: {'model__max_depth': 20, 'model__min_samples_leaf': 2, 'model__min_samples_split': 2, 'model__n_estimators': 300}
CV-RMSE RF: 518.7404923642932


In [132]:
best_rf = rf_grid.best_estimator_

y_pred_test_rf = best_rf.predict(X_test)
rf_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test_rf))
print("Test-RMSE RF:", rf_rmse_test)

Test-RMSE RF: 471.195082260569


Ridge gav ungefär samma resultat som de andra två linjära modellerna. RandomForestRegressor tog en bra stund så jag fick förhoppningar, men de kom på skam. :(

SVR verkar ha fått bäst resultat i gruppen. Tar en liten titt. Vi börjar med en linjär modell.

In [148]:

# Pipeline
svr_pipe = Pipeline([
    ("scaler", StandardScaler ()),
    ("model", SVR(kernel="linear"))
])

# Param grid 
param_grid_svr = {
    "model__C": [100, 300, 500],
    "model__epsilon": [10, 50, 100]
}

# CV 
svr_grid_linear = GridSearchCV(
    svr_pipe,
    param_grid_svr,
    scoring="neg_root_mean_squared_error",
    cv=5,  
    n_jobs=-1,
    verbose = 2
)

# Träna
svr_grid_linear.fit(X_trainval, y_trainval)

# CV-resultat
print("Bästa hyperparametrar SVR (linear):", svr_grid_linear.best_params_)
print("CV-RMSE SVR:", -svr_grid_linear.best_score_)




Fitting 5 folds for each of 9 candidates, totalling 45 fits
Bästa hyperparametrar SVR (linear): {'model__C': 500, 'model__epsilon': 50}
CV-RMSE SVR: 88.91309744401451


In [ ]:
# Bästa estimator
best_svr = svr_grid_linear.best_estimator_

# Kör mot testdata
y_pred_test_svr = best_svr.predict(X_test)
svr_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test_svr))
print("Test-RMSE SVR:", svr_rmse_test)

Test-RMSE SVR: 64.55582785909775


I testen av SVR upptäckte vi att det var en dålig idé att slarva med hyperparameterar i GridSearchCV. Det blev oerhört tungt för datorn att tugga igenom med fel värden och först efter ett antal experiment visade det sig att ett för högt C-värde och för många parametrar tog väldigt lång tid.   

Vi tar en ny SVR-modell

In [149]:

# Pipeline
svr_pipe = Pipeline([
    ("scaler", StandardScaler ()),
    ("model", SVR(kernel="poly"))
])

# Param grid 
param_grid_svr = {
    "model__C": [10, 300],
    "model__epsilon": [0.1, 0.5],
    "model__degree": [2, 3]
}

# CV 
svr_grid_poly = GridSearchCV(
    svr_pipe,
    param_grid_svr,
    scoring="neg_root_mean_squared_error",
    cv=5,  
    n_jobs=-1,
    verbose = 3
)

# Träna
svr_grid_poly.fit(X_trainval, y_trainval)

# CV-resultat
print("Bästa hyperparametrar SVR (poly):", svr_grid_poly.best_params_)
print("CV-RMSE SVR:", -svr_grid_poly.best_score_)




Fitting 5 folds for each of 8 candidates, totalling 40 fits
Bästa hyperparametrar SVR (poly): {'model__C': 300, 'model__degree': 3, 'model__epsilon': 0.1}
CV-RMSE SVR: 587.412053225681


In [ ]:
# Bästa estimator
best_svr = svr_grid_poly.best_estimator_


# Kör mot testdata
y_pred_test_svr = best_svr.predict(X_test)
svr_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test_svr))
print("Test-RMSE SVR:", svr_rmse_test)

Test-RMSE SVR: 64.55582785909775


En sista SVR. Stötte på patrull i jämförelse med Linus och gör ett nytt försök från grunden.       

In [151]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

# 1) Läs in data på nytt
df2 = pd.read_csv("car_price_dataset.csv", sep=";")

# 2) Features och target (samma som Linus)
features = ["Model", "Year", "Engine_Size", "Fuel_Type", "Transmission", "Mileage"]
X2 = df2[features]
y2 = df2["Price"]

# 3) Train/val + test-split (20 % test)
X2_trainval, X2_test, y2_trainval, y2_test = train_test_split(
    X2, y2, test_size=0.2, random_state=42
)

# 4) Preprocessor (som Linus)
numeric_features = ["Year", "Mileage", "Engine_Size"]
categorical_features = ["Model", "Fuel_Type", "Transmission"]

preprocessor2 = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

# 5) Pipeline med rbf-SVR
svr_model2 = Pipeline([
    ("preprocessor", preprocessor2),
    ("model", SVR(kernel="rbf"))
])

# 6) Param grid (din rbf-grid, nu på rådata + preprocessor)
param_grid_svr2 = {
    "model__C": [100, 300, 1000],
    "model__epsilon": [0.01, 0.1, 0.5],
    "model__gamma": [0.03, 0.05, 0.1]
}

# 7) CV (5-fold, med shuffle för att matcha Linus bättre)
cv2 = KFold(n_splits=5, shuffle=True, random_state=42)

svr_grid_rbf2 = GridSearchCV(
    estimator=svr_model2,
    param_grid=param_grid_svr2,
    scoring="neg_root_mean_squared_error",
    cv=cv2,
    n_jobs=-1,
    verbose=3
)

# 8) Träna på train/val
svr_grid_rbf2.fit(X2_trainval, y2_trainval)

# 9) CV-resultat
print("Bästa hyperparametrar SVR(rbf):", svr_grid_rbf2.best_params_)
print("CV-RMSE SVR (train/val):", -svr_grid_rbf2.best_score_)

# (valfritt) Test-RMSE på X2_test / y2_test
best_svr_rbf2 = svr_grid_rbf2.best_estimator_
y2_pred_test = best_svr_rbf2.predict(X2_test)
rmse_test2 = np.sqrt(mean_squared_error(y2_test, y2_pred_test))
print("Test-RMSE SVR (rbf):", rmse_test2)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Bästa hyperparametrar SVR(rbf): {'model__C': 1000, 'model__epsilon': 0.5, 'model__gamma': 0.1}
CV-RMSE SVR (train/val): 65.02258085477962
Test-RMSE SVR (rbf): 50.292764897579346
